## This notebook is the initial replica of running several models to the calibrations parameters for EcoTwins.

Importing the requeried libraries

In [3]:
os.getcwd()

'/Users/fbenitez/PycharmProjects/RAMP-UA_EcoTwins1709/experiments/calibration'

In [1]:
import numpy as np
import yaml # pyyaml library for reading the parameters.yml file
import os
import itertools

import matplotlib.pyplot as plt

from model.opencl.ramp import run_headless
from microsim.opencl.ramp.snapshot_convertor import SnapshotConvertor
from microsim.opencl.ramp.snapshot import Snapshot
from microsim.opencl.ramp.params import Params, IndividualHazardMultipliers, LocationHazardMultipliers
from microsim.opencl.ramp.simulator import Simulator
from microsim.opencl.ramp.disease_statuses import DiseaseStatus

# Useful for connecting to this kernel
%connect_info


ModuleNotFoundError: No module named 'microsim'